In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#setup
cw = pd.read_csv("../input/world-university-rankings/cwurData.csv",na_values=['-'])
cw = cw.drop_duplicates()
year11 = cw['year']
year22 = year11 == 2015
cw1 = cw[year22]
cw1=cw1.dropna()
cw2=cw1[['institution','country','world_rank','score']]
cw2=cw2.rename(columns={'institution': "Institution",'country': 'Country',
                        'world_rank':'CWUR_World_Rank','score':'CWUR_Score'})
cw2
sh = pd.read_csv("../input/world-university-rankings/shanghaiData.csv",na_values=['-'])
sh = sh.drop_duplicates()
sh['score'] =  0.1 * sh['alumni'] + 0.2 * sh['award'] + 0.2 * sh['hici'] + 0.2 * sh['ns'] + 0.2 * sh['pub'] + 0.1 * sh['pcp']
sh.total_score.fillna(sh.score, inplace=True)
del sh['score']
year1 = sh['year']
year2 = year1 == 2015
sh1 = sh[year2]
sh1=sh1.dropna()
sh2=sh1[['university_name','world_rank','total_score']]
sh2=sh2.rename(columns={'university_name': "Institution",'world_rank':'SH_World_Rank',
                        'total_score':'SH_Score'})
sh2
time = pd.read_csv("../input/world-university-rankings/timesData.csv",na_values=['-'])
time = time.drop_duplicates()
time['score'] =  0.3 * time['teaching'] + 0.075 * time['international'] + 0.3 * time['research'] + 0.3 * time['citations'] + 0.025 * time['income']
time.total_score.fillna(time.score, inplace=True)
del time['score']

year111 = time['year']
year222 = year111 == 2015
time1 = time[year222]
time2=time1[['university_name','world_rank','total_score','student_staff_ratio']]
time2=time2.rename(columns={'university_name': "Institution",'world_rank':'Times_World_Rank',
                            'total_score':'Times_Score','student_staff_ratio':'Student_Staff_Ratio'})
time2
final=pd.merge(pd.merge(cw2,sh2,on='Institution'),time2,on='Institution')
df=final.head(100)
df['Overall_Score'] = 1/3*(df['CWUR_Score']+df['SH_Score']+df['Times_Score'])
df['Overall_Rank'] = df['Overall_Score'].rank(ascending = 0) 

In [ ]:
exp=pd.read_csv("../input/world-university-rankings/education_expenditure_supplementary_data.csv",na_values=['-'],sep='\t', lineterminator='\r')
exp = exp.drop_duplicates()
split_exp=exp['country,institute_type,direct_expenditure_type,1995,2000,2005,2009,2010,2011'].str.split(',')
exp
exp1= split_exp.to_list()
names = ['country','institute_type','direct_expenditure_type','1995','2000','2005','2009','2010','2011']
new_exp = pd.DataFrame(exp1,columns=names)
new_exp

In [ ]:
cw.corr()
import matplotlib.pyplot as plt
import seaborn as sns 
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(cw.corr(), annot=True, linewidths=.1, fmt= '.1f',ax=ax)
plt.show()